In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

2023-04-21 22:50:20.592637: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 22:50:21.122356: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
os.chdir("../../")

In [3]:
from programs import lstm_tool

2023-04-21 22:50:22.381985: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
pre_trained_predicter = tf.keras.models.load_model('./model/power_lstm')
compare_predicter = tf.keras.models.load_model('./model/power_lstm')
checker = tf.keras.models.load_model('./model/checker.h5')

2023-04-21 22:50:22.684999: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-04-21 22:50:22.685095: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-04-21 22:50:22.685152: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [5]:
df = pd.read_excel('./documents/nov_nine_var.xlsx').iloc[:,1:22].to_numpy()
scaler = MinMaxScaler()
X = scaler.fit_transform(df)

In [6]:
x, y = lstm_tool.make_sequence(X, 60)

In [7]:
# 예측 -> 확인
# case 0:
#   train
# case 1:
#   recur


In [8]:
y[0]

array([0.94736842, 0.19912861, 0.95490972, 0.16190806, 0.34488938,
       0.30802127, 0.87429939, 0.18766606, 0.49327062, 0.02481508,
       0.98095238, 0.90225564, 0.42261905, 1.        , 0.46268657,
       0.9893617 , 0.14629913, 1.        , 0.05292048, 0.06728395,
       0.07137273])

In [9]:
pre_trained_predicter(x[0:1])

2023-04-21 22:50:26.863039: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [1,60,21]
	 [[{{node inputs}}]]


<tf.Tensor: shape=(1, 21), dtype=float32, numpy=
array([[0.9479111 , 0.20294794, 0.9372454 , 0.17837885, 0.36683658,
        0.3039113 , 0.66569185, 0.2855119 , 0.6535803 , 0.04525163,
        0.97602856, 0.8972882 , 0.4038009 , 0.99818593, 0.47038192,
        0.98779976, 0.16106793, 0.9846732 , 0.07167893, 0.08588298,
        0.07903786]], dtype=float32)>

In [10]:
checker([x[0:1], pre_trained_predicter(x[0:1])])

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.8444599]], dtype=float32)>

In [11]:
x[0:1].shape

(1, 60, 21)

In [12]:
predict_period = X.shape[0]

In [13]:
i = 0
inp = x[0:1]

while True:
    with tf.GradientTape() as tape:
        outp = pre_trained_predicter(inp)
        check = checker([inp, outp])

        loss = tf.losses.binary_crossentropy(tf.ones_like(check), check)
    
    t = check[0][0]
    if t < 0.80:
        print(i, t)
        grads = tape.gradient(loss, pre_trained_predicter.trainable_weights)
        pre_trained_predicter.optimizer.apply_gradients(zip(grads, pre_trained_predicter.trainable_weights))
        
        inp = x[0:1]
        i = 0

    else:
        inp = lstm_tool.shift_data(inp, outp)
        i += 1

    if i == predict_period:
        break

2023-04-21 22:50:27.314791: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/lstm_1/PartitionedCall_grad/lstm_1/PartitionedCall' with dtype float and shape [1,?,32]
	 [[{{node gradients/lstm_1/PartitionedCall_grad/lstm_1/PartitionedCall}}]]
2023-04-21 22:50:27.314872: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/lstm_1/PartitionedCall_grad/lstm_1/PartitionedCall_1' with dtype int32 and shape [3]
	 [[{{node gradients/lstm_1/PartitionedCall_grad/lstm_1/PartitionedCall_1}}]]
2023-04-21 22:50:27.314925: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start abor

21 tf.Tensor(0.7981409, shape=(), dtype=float32)


KeyboardInterrupt: 

In [ ]:
recursive_data = lstm_tool.plot_tree([pre_trained_predicter], x[0:1], 1, predict_period, True)

In [ ]:
compare_data = lstm_tool.plot_tree([compare_predicter], x[0:1], 1, predict_period, True)
plt.plot(lstm_tool.dnn_model(X))
plt.show()

In [ ]:
for i in range(len(compare_data)-61):
    cinp = compare_data[i:i+60].reshape(1, 60, 21)
    coutp = compare_data[i+60+1].reshape(1, 21)
    
    tinp = recursive_data[i:i+60].reshape(1, 60, 21)
    toutp = recursive_data[i+60+1].reshape(1, 21)
    
    a = int(round(checker([tinp, toutp]), 1))
    b = int(round(checker([cinp, coutp]), 1))
    if a != b:
        print(i, " : ", a, b)